# Task3 

Build a matrix of audiences (by X RU_platform, by Y - RU_age), age values - 10% percentiles. Fill in the received matrix: 
1) Number of RUs 

2) Percentage of RU from the total 

3) retention of the 1st day

### 3.1

```
with reg as(
    select
        ceil(percent_rank() over(order by age)*10) as percentile,
        platform,
        count(*) over() as total
    from 'regular_users'
    where true
    and (date_registration between '2018-09-15' and '2019-01-29'
)
    
select *
from reg
pivot(max(total) for platform in ('mobile', 'desktop', 'tablet', 'tv')
     )
order by percentile
```

### 3.2

```
with reg as(
    select
        ceil(percent_rank() over(order by age)*10) as percentile,
        platform,
        count(*) over() as total
    from 'regular_users'
    where true
    and (date_registration between '2018-09-15' and '2019-01-29'
),

to_count as(
    select
        percentile,
        platform,
        round(count(*) / max(total)*100, 2) as perc
    from reg
    group by 1, 2
    order by 1, 2
)
    
select *
from reg
pivot(max(perc) for platform in ('mobile', 'desktop', 'tablet', 'tv')
     )
order by percentile
```

### 3.3

```
with reg as(
    select
        id_ru,
        date_registration,
        ceil(percent_rank() over(order by age)*10) as percentile,
        platform
    from 'regular_users'
    where true
    and (date_registration between '2018-09-15' and '2019-01-29'
),
    
logins as(
    select
        id_ru,
    date_login,
    from 'logins'
    where true
        and (date_login between '2018-09-15' and '2019-01-30')
),
    
all_retention as (
    select
        id_ru,
        max(if(date_diff(date_login, date_registration, day) = 1, 1, 0)) as reten
    from reg
        left join logins using(id_ru)
    group by 1
),    
    
percen as (
    select   
        if(percentile < 1, 1, percentile) as percentile,
        platform,
        round(sum(reten) / count(reten)* 100, 2) as percent
    from reg
        left join all_retention using(id_ru)
    group by 1, 2
    order by 1, 2
)    
    
select *
from percen
    pivot(max(percent) for platform in ('mobile', 'desktop', 'tablet', 'tv') 
         )
order by percentile
```